In [ ]:
!pip3 install gdown

In [ ]:
!pip install tensorflow-text

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import lightgbm as lgbm

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

from tqdm import tqdm #progress bar


In [ ]:
!gdown https://drive.google.com/uc?id=1fxp4rETrjg3_xbMqCRAGe1PE5pDa11Ps

Downloading...
From: https://drive.google.com/uc?id=1fxp4rETrjg3_xbMqCRAGe1PE5pDa11Ps
To: /content/combined_datasets.zip
100% 305M/305M [00:04<00:00, 71.1MB/s]


In [ ]:
!unzip combined_datasets.zip

Archive:  combined_datasets.zip
   creating: combined_datasets/
  inflating: combined_datasets/.DS_Store  
  inflating: __MACOSX/combined_datasets/._.DS_Store  
  inflating: combined_datasets/combined.csv  
  inflating: combined_datasets/balanced.csv  


In [ ]:
%cd combined_datasets/

/content/combined_datasets


In [ ]:
import pandas as pd


In [ ]:
df_balanced = pd.read_csv("balanced.csv")
df_combined = pd.read_csv("combined.csv")

In [ ]:
df_balanced

,comment_text,toxicity
0,Far too many people see hunters as city folks ...,0
1,Sadly it will not happen because the politiciz...,0
2,You are correct. Thank you.,0
3,As long as that person's gender identity is no...,0
4,Jackass,1
...,...,...
274705,It's not good enough. What arête hey waiting f...,0
274706,"Yes, I get a kick when a high school dropout t...",1
274707,Another case of Legislators involving themselv...,0
274708,you just showed that you base your decisions u...,0


In [ ]:
df_combined

,comment_text,toxicity
0,"""Make a mess"".",0
1,"It,s made in London where the liberals have tw...",0
2,:::::The proposal was passed by an overwhelm...,0
3,"""Just fascinating! It has probably been answer...",0
4,Your comment is one of the problems with Eugen...,0
...,...,...
2085484,71% of Canadians do not agree with PM Gomer. F...,0
2085485,"The law needs to be repealed, what a ridiculou...",1
2085486,excellent use of the pastime to cast introspec...,0
2085487,When I look closely at your writing I see the ...,0


In [ ]:
!pip install catboost

import catboost as cb

import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV



     |████████████████████████████████| 76.3 MB 1.3 MB/s 


In [ ]:

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from lightgbm import LGBMClassifier

In [ ]:
# train_df, X_test, y_train, y_test= train_test_split(df_balanced.comment_text, df_balanced.toxicity, test_size=0.2, random_state=123)
#train_df, val_df= train_test_split(df_balanced[:20000], test_size=0.2, random_state=123)
train_df, val_df= train_test_split(df_combined, test_size=0.2, random_state=123)


In [ ]:
# Load encoder from Tensorflow for LGBM model
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

# Embed sentences for the training set
X_train = []
for r in tqdm(train_df.comment_text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)


 12%|█▏        | 194887/1668391 [2:03:19<16:46:18, 24.40it/s]

In [ ]:
y_train = train_df.toxicity.values

# Embed sentences for the validation set
Val_test = []
for r in tqdm(val_df.comment_text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  Val_test.append(review_emb)


In [ ]:
# LGBM

# Split data into train and testing data
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.1, random_state = 42)

# Get the train and test data for the training sequence
train_data = lgbm.Dataset(X_train, label=y_train)
test_data = lgbm.Dataset(X_test, label=y_test)

# Parameters we'll use for the prediction
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'boosting': 'dart',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [ ]:
# Train the classifier
classifier = lgbm.train(parameters,
                       train_data,
                       valid_sets= test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

# PREDICTION
val_pred = classifier.predict(Val_test)


In [ ]:
len(val_pred)

In [ ]:
len(Val_test)

In [ ]:
val_pred = val_pred.round().astype(int)

In [ ]:
type(Val_test)

In [ ]:
import sklearn.metrics as metrics


In [ ]:
print("Accuracy Score ->",accuracy_score(val_pred, val_df.toxicity.values)*100)

In [ ]:
print(metrics.confusion_matrix(val_df.toxicity.values, val_pred, labels=[0, 1]))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(val_df.toxicity.values, val_pred, labels=[0, 1]))


In [ ]:
def predictSentence(inputArr):
  testing_Sentences = []
  for r in inputArr:
    emb = use(r)
    review_emb = tf.reshape(emb, [-1]).numpy()
    testing_Sentences.append(review_emb)

  testing_Sentences = np.array(testing_Sentences)

  result = classifier.predict(testing_Sentences)
  #print(type((result[0]*100).round().astype(int)))
  print("The toxicity score for the sentence is: {} ".format(result))

In [ ]:
inputArr = ["I hate american people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Black people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Asian people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Japanese people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Korean people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Chinese people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Finnish people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate European people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate American people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Indian people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate African people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Eastern people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate Western people"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate black"]
predictSentence(inputArr)

In [ ]:
inputArr = ["I hate white"]
predictSentence(inputArr)